# Task 1 - Context

In [2]:

import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

seed = 42


# Task 2 - The Data

Dataset information

Network was collected by crawling Amazon website. It is based on Customers Who Bought This Item Also Bought feature of the Amazon website. If a product i is frequently co-purchased with product j, the graph contains an undirected edge from i to j. Each product category provided by Amazon defines each ground-truth community.

We regard each connected component in a product category as a separate ground-truth community. We remove the ground-truth communities which have less than 3 nodes. We also provide the top 5,000 communities with highest quality which are described in our paper. As for the network, we provide the largest connected component. 

In [3]:
file_path = Path(os.path.join('..', 'data', 'com-amazon.ungraph.txt.gz'))
df_graph = pd.read_csv(file_path, compression='gzip', delimiter='\t', skiprows=3).rename(columns={'# FromNodeId': 'FromNodeId', 'ToNodeId': 'ToNode'})
display(df_graph.head())

,FromNodeId,ToNode
0,1,88160
1,1,118052
2,1,161555
3,1,244916
4,1,346495


# Task 3 – IDA and Preprocessing

In [6]:
G = nx.from_pandas_edgelist(df_graph, 'FromNodeId', 'ToNode')


print(f'Number of Nodes: {G.size()}')
print(f'Number of Edges: {G.order()}')

Number of Nodes: 925872
Number of Edges: 334863


In [5]:
# pos = nx.spring_layout(G, seed=42)
# # Graphen plotten
# plt.figure(figsize=(8, 6))
# nx.draw(G=G,pos=pos, with_labels=True, node_size=700, node_color='skyblue', font_size=15, font_color='black', font_weight='bold')
# plt.title('Graph Plot')
# plt.show()

# Task 4 – Graph Properties

# Task 5 – Central Nodes

# Task 6 – Node Roles

# Task 7 – Community Discovery

# Task 8 – Conclusions and Future Work